In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error


In [24]:
train_data=pd.read_csv('cleaned_data_set/cleaned.csv')

In [25]:
X=train_data.drop(['pedal_cycles','two_wheeled_motor_vehicles','cars_and_taxis','buses_and_coaches','lgvs','all_hgvs','all_motor_vehicles'],axis=1)
y=train_data['two_wheeled_motor_vehicles']

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [27]:
scaler = MinMaxScaler()

In [28]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [29]:
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

LSTM

In [30]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

In [31]:
lstm_model.fit(X_train_reshaped, y_train, epochs=20, batch_size=32)

Epoch 1/20
21665/21665 [==============================] - 27s 1ms/step - loss: 186.3385
Epoch 2/20
21665/21665 [==============================] - 60s 3ms/step - loss: 177.0814
Epoch 3/20
21665/21665 [==============================] - 48s 2ms/step - loss: 171.7743
Epoch 4/20
21665/21665 [==============================] - 28s 1ms/step - loss: 169.4678
Epoch 5/20
21665/21665 [==============================] - 29s 1ms/step - loss: 168.7026
Epoch 6/20
21665/21665 [==============================] - 28s 1ms/step - loss: 168.4425
Epoch 7/20
21665/21665 [==============================] - 27s 1ms/step - loss: 168.3108
Epoch 8/20
21665/21665 [==============================] - 27s 1ms/step - loss: 168.2138
Epoch 9/20
21665/21665 [==============================] - 27s 1ms/step - loss: 168.1569
Epoch 10/20
21665/21665 [==============================] - 29s 1ms/step - loss: 168.0764
Epoch 11/20
21665/21665 [==============================] - 26s 1ms/step - loss: 168.0725
Epoch 12/20
21665/21665 [=====

In [32]:
y_pred_lstm = lstm_model.predict(X_test_reshaped)
mse_lstm = mean_squared_error(y_test, y_pred_lstm)
r2_lstm = r2_score(y_test, y_pred_lstm)

5417/5417 [==============================] - 14s 2ms/step


In [33]:
y_pred_lstm = lstm_model.predict(X_test_reshaped)
mse_lstm = mean_squared_error(y_test, y_pred_lstm)
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)
rmse_lstm= np.sqrt(mse_lstm)
print(f"LSTM Performance: MSE = {mse_lstm:.2f}, MAE Score = {mae_lstm:.2f}, RMSE Score = {rmse_lstm:2f}")


5417/5417 [==============================] - 12s 2ms/step
LSTM Performance: MSE = 165.16, MAE Score = 5.14, RMSE Score = 12.851561


In [34]:
lstm_model.save('models/two_wheeled_lstm.keras')

In [35]:
gru_model = Sequential()
gru_model.add(GRU(50, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
gru_model.add(Dense(1))
gru_model.compile(optimizer='adam', loss='mse')

In [36]:
gru_model.fit(X_train_reshaped, y_train, epochs=30, batch_size=32)

Epoch 1/30
21665/21665 [==============================] - 48s 2ms/step - loss: 184.9023
Epoch 2/30
21665/21665 [==============================] - 30s 1ms/step - loss: 175.8156
Epoch 3/30
21665/21665 [==============================] - 30s 1ms/step - loss: 171.1520
Epoch 4/30
21665/21665 [==============================] - 29s 1ms/step - loss: 169.4172
Epoch 5/30
21665/21665 [==============================] - 28s 1ms/step - loss: 168.8431
Epoch 6/30
21665/21665 [==============================] - 29s 1ms/step - loss: 168.5284
Epoch 7/30
21665/21665 [==============================] - 28s 1ms/step - loss: 168.2599
Epoch 8/30
21665/21665 [==============================] - 50s 2ms/step - loss: 168.1307
Epoch 9/30
21665/21665 [==============================] - 26s 1ms/step - loss: 168.0594
Epoch 10/30
21665/21665 [==============================] - 28s 1ms/step - loss: 167.9688
Epoch 11/30
21665/21665 [==============================] - 27s 1ms/step - loss: 167.8878
Epoch 12/30
21665/21665 [=====

In [37]:
y_pred_gru = gru_model.predict(X_test_reshaped)
mse_gru = mean_squared_error(y_test, y_pred_gru)
mae_gru = mean_absolute_error(y_test, y_pred_gru)
rmse_gru = np.sqrt(mse_gru)

5417/5417 [==============================] - 6s 1ms/step


In [38]:
print(f"GRU Performance for pedal cycles: MSE = {mse_gru:.2f}, MAE Score = {mae_gru:.2f}, RMSE Score = {rmse_gru:2f}")


GRU Performance for pedal cycles: MSE = 164.43, MAE Score = 5.46, RMSE Score = 12.823061


In [39]:
gru_model.save('models/two_wheeled_gru.keras')

In [40]:
y_train = np.array(y_train)
svr = LinearSVR(max_iter=5, random_state=42)
if y_train.ndim == 1:  # Check if it's 1D
    y_train = y_train.reshape(-1, 1)

svm_model = MultiOutputRegressor(svr, n_jobs=-1)
svm_model.fit(X_train, y_train)

MultiOutputRegressor(estimator=LinearSVR(max_iter=5, random_state=42),
                     n_jobs=-1)

In [41]:
joblib.dump(svm_model, f'models/two_wheeled_svm.pkl')

['models/two_wheeled_svm.pkl']

In [42]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

y_pred = svm_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"SVR Performance for pedal cycle: MSE = {mse:.2f}, MAE Score = {mae:.2f}, RMSE= {rmse:.2f}")

SVR Performance for pedal cycle: MSE = 203.25, MAE Score = 5.03, RMSE= 14.26
